[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/see/transduction_bodies/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [1]:
# Preparation

# install tensor2tensor
! pip install -q -U tensor2tensor

# get the needed resources
! curl -O http://josd.github.io/eye/see/transduction_bodies/observation_prediction_bodies.sh
! curl -O http://josd.github.io/eye/see/transduction_bodies/observation_prediction_bodies.py
! curl -O http://josd.github.io/eye/see/transduction_bodies/__init__.py
! curl -O http://josd.github.io/eye/see/transduction_bodies/test_bodies.observation
! chmod +x observation_prediction_bodies.sh

# clear data and model
% rm -fr /tmp/t2t_data/observation_prediction_bodies/
% rm -fr /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/

# start tensorboard
! curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1311  100  1311    0     0   1311      0  0:00:01 --:--:--  0:00:01  6900
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2244  100  2244    0     0   2244      0  0:00:01 --:--:--  0:00:01 10840
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    44  100    44    0     0     44      0  0:00:01 --:--:--  0:00:01   235
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    109      0  0:00:01 --:--:--  0:00:01   586
  % Total    % Received % Xferd  Average Speed   Tim

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      # wind turbine size factor
      size_factor = 

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=/tmp/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=/tmp/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# train with Adam for 4000 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=4000

# train with SGD for 1000 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams="optimizer=SGD" \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=5000

# decode
t2t-decoder \
  --data_dir=$

In [5]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

INFO:tensorflow:Importing user module content from path /
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd058a8beb8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow

INFO:tensorflow:Finished evaluation at 2018-07-05-09:30:49
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 0.0064503746, metrics-observation_prediction_bodies/targets/accuracy = 0.9987302, metrics-observation_prediction_bodies/targets/accuracy_per_sequence = 0.9898413, metrics-observation_prediction_bodies/targets/accuracy_top5 = 1.0, metrics-observation_prediction_bodies/targets/approx_bleu_score = 0.99709386, metrics-observation_prediction_bodies/targets/neg_log_perplexity = -0.08979906, metrics-observation_prediction_bodies/targets/rouge_2_fscore = 0.9970976, metrics-observation_prediction_bodies/targets/rouge_L_fscore = 0.9987302
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt-5000
INFO:tensorflow:Reading data files from /tmp/t2t_data/observation_prediction_bodies/observation_prediction_bodies-dev*
INFO:tensorflow:partition: 0 num_data_files: 3
I

INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-05-09:31:24
INFO:tensorflow:Graph was finalized.
2018-07-05 09:31:24.764272: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-07-05 09:31:24.764813: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-07-05 09:31:24.764858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-05 09:31:25.165231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device

INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-05-09:31:42
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 0.006450375, metrics-observation_prediction_bodies/targets/accuracy = 0.9987302, metrics-observation_prediction_bodies/targets/accuracy_per_sequence = 0.9898413, metrics-observation_prediction_bodies/targets/accuracy_top5 = 1.0, metrics-observation_prediction_bodies/targets/approx_bleu_score = 0.99709386, metrics-observation_prediction_bodies/targets/neg_log_perplexity = -0.08979906, metrics-observation_prediction_bodies/targets/rouge_2_fscore = 0.9970976, metrics-observation_prediction_bodies/targets/rouge_L_fscore = 0.9987302
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt-5000
INFO:tensorflow:Importing user 

In [6]:
# See the transductions
# For each observation the top 2 predictions are shown with their respective log probability

! pygmentize -g test_bodies.observation
print("->-")
! pygmentize -g test_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.43	A_PERSON has BMI class U	-4.71
A_TURBINE producing 9533 kW	-0.39	A_TURBINE producing 12577 kW	-4.36


In [7]:
import os

import tensorflow as tf

from tensor2tensor import problems
from tensor2tensor.bin import t2t_decoder  # To register the hparams set
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.visualization import attention
from tensor2tensor.visualization import visualization

In [8]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [9]:
# MODEL
CHECKPOINT = os.path.expanduser('/tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small')

In [10]:
# HPARAMS
problem_name = 'observation_prediction_bodies'
data_dir = os.path.expanduser('/tmp/t2t_data/observation_prediction_bodies')
model_name = "transformer"
hparams_set = "transformer_small"

In [11]:
import observation_prediction_bodies

visualizer = visualization.AttentionVisualizer(hparams_set, model_name, data_dir, problem_name, beam_size=1)

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Greedy Decoding


In [12]:
tf.Variable(0, dtype=tf.int64, trainable=False, name='global_step')

sess = tf.train.MonitoredTrainingSession(
    checkpoint_dir=CHECKPOINT,
    save_summaries_secs=0,
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.


In [13]:
input_sentence = "A_TURBINE with size factor 4 and subjected to windspeed 62 km/h"
output_string, inp_text, out_text, att_mats = visualizer.get_vis_data_from_string(sess, input_sentence)
print(output_string)

A_TURBINE producing 9533 kW<EOS>


## Interpreting the Visualizations
- The layers drop down allow you to view the different Transformer layers, 0-indexed of course.
  - Tip: The first layer, last layer and 2nd to last layer are usually the most interpretable.
- The attention dropdown allows you to select different pairs of encoder-decoder attentions:
  - All: Shows all types of attentions together. NOTE: There is no relation between heads of the same color - between the decoder self attention and decoder-encoder attention since they do not share parameters.
  - Input - Input: Shows only the encoder self-attention.
  - Input - Output: Shows the decoder’s attention on the encoder. NOTE: Every decoder layer attends to the final layer of encoder so the visualization will show the attention on the final encoder layer regardless of what layer is selected in the drop down.
  - Output - Output: Shows only the decoder self-attention. NOTE: The visualization might be slightly misleading in the first layer since the text shown is the target of the decoder, the input to the decoder at layer 0 is this text with a GO symbol prepreded.
- The colored squares represent the different attention heads.
  - You can hide or show a given head by clicking on it’s color.
  - Double clicking a color will hide all other colors, double clicking on a color when it’s the only head showing will show all the heads again.
- You can hover over a word to see the individual attention weights for just that position.
  - Hovering over the words on the left will show what that position attended to.
  - Hovering over the words on the right will show what positions attended to it.

In [14]:
call_html()
attention.show(inp_text, out_text, *att_mats)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>